In [1]:
pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
import cv2
import os
import numpy as np
from PIL import Image

In [3]:
# 1. Dataset Class for Water Meter Digits
class WaterMeterDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.images = []
        self.labels = []
        
        for label in range(10):  # Assuming folder structure: image_folder/0, image_folder/1, ..., image_folder/9
            digit_folder = os.path.join(image_folder, str(label))
            if not os.path.exists(digit_folder):
                continue
            for img_name in os.listdir(digit_folder):
                img_path = os.path.join(digit_folder, img_name)
                self.images.append(img_path)
                self.labels.append(label)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [4]:
# 2. Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [13]:
# 3. Load Dataset
train_dataset = WaterMeterDataset('digits updated', transform=transform)
val_dataset = WaterMeterDataset('digits_jpeg', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [14]:
# 4. Load Pretrained ResNet-18
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(512, 10)  # Replace final layer for digit classification
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=model.to(device)

In [15]:
# 5. Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [16]:
# 6. Training Loop
def train_model(model, train_loader, val_loader, epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")
        validate_model(model, val_loader)

In [17]:
# 7. Validation
def validate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Validation Accuracy: {100 * correct / total:.2f}%')

In [18]:
train_model(model, train_loader, val_loader, epochs=5)

Epoch 1, Loss: 0.07409259209112094
Validation Accuracy: 98.73%
Epoch 2, Loss: 0.01739268904172873
Validation Accuracy: 100.00%
Epoch 3, Loss: 0.01479630269252839
Validation Accuracy: 96.07%
Epoch 4, Loss: 0.012295131358528897
Validation Accuracy: 100.00%
Epoch 5, Loss: 0.01884148750573989
Validation Accuracy: 99.93%


In [19]:
# 8. Save Model
torch.save(model.state_dict(), 'water_meter_resnet18.pth')

In [22]:
# 9. Load and Test on a New Image
def predict_digit(model, image_path):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    
    return predicted.item()

In [23]:
# Test on a new image
print("Predicted Digit:", predict_digit(model, 'Untitled Folder/image/id_1_value_13_116.jpg'))

Predicted Digit: 7
